In [0]:
%run ./Shared_Functions

In [0]:
# COMMAND ----------
# 0. SETUP & UTILITIES
# MAGIC %run ./Shared_Functions

# COMMAND ----------
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType

In [0]:
# 1. GET RUN ID & WIDGETS
try:
    run_id = dbutils.notebook.entry_point.getDbutils().notebook().getContext().runId().get()
except:
    run_id = "manual_run"

dbutils.widgets.text("process_type", "all")
process_type = dbutils.widgets.get("process_type")

print(f"Starting Silver Transformation for mode: {process_type} | Run ID: {run_id}")

In [0]:
# ==========================================
# PATH A: TRANSFORM USERS (SCD Type 2 Version)
# ==========================================
if process_type in ["users", "all"]:
    job_name = "Silver_Transform_Users"
    try:
        print(f">>> [START] {job_name}...")
        
        # 1. READ FROM BRONZE
        usersDF = spark.read.table("ecomm_data_project.bronze.users")
        
        # 2. TRANSFORMATION LOGIC
        users_silver = (
            usersDF
            .withColumn("countrycode", F.upper(F.col("countrycode")))
            .withColumn(
                "gender",
                F.when(F.col("gender").startswith("M"), "Male")
                .when(F.col("gender").startswith("F"), "Female")
                .otherwise("Other")
            )
            .withColumn("civilitytitle_clean", F.regexp_replace("civilitytitle", "(Mme|Ms|Mrs)", "Ms"))
            .withColumn(
                "language_full",
                F.expr(
                    "CASE WHEN language = 'EN' THEN 'English' "
                    "WHEN language = 'FR' THEN 'French' "
                    "ELSE 'Other' END"
                )
            )
            .withColumn("account_age_years", F.round(F.col("seniority") / 365, 2))
            .withColumn("productsSold", F.col("productsSold").cast("int"))
            .withColumn("productsWished", F.col("productsWished").cast("int"))
            .filter(F.col("identifierHash").isNotNull())
            .dropDuplicates(["identifierHash"])
        )

        # 3. ADD SCD TYPE 2 METADATA (Alignment with Perfect Shell)
        # We add the columns we defined in the 'CREATE TABLE' statement
        updates_df = users_silver.withColumn("is_current", F.lit(True)) \
                                 .withColumn("effective_start_date", F.current_timestamp()) \
                                 .withColumn("effective_end_date", F.lit(None).cast("timestamp"))

        # 4. COLUMN ORDERING (Critical for Managed Tables)
        # We must match the exact order of the columns in your '00_Setup_Infrastructure'
        column_order = [
        "identifierHash", "countrycode", "gender", "civilitytitle_clean", 
        "language_full", "account_age_years", 
        "productsSold", "productsWished", # <--- Add these two here!
        "is_current", "effective_start_date", "effective_end_date"
        ]
        updates_df = updates_df.select(*column_order)

        # 5. PERFORM MERGE / UPSERT LOGIC
        target_table = "ecomm_data_project.silver.users"
        
        # We check if data already exists to handle history
        if spark.catalog.tableExists(target_table) and spark.read.table(target_table).count() > 0:
            print(">>> Table has data. Processing history...")
            from delta.tables import DeltaTable
            dt = DeltaTable.forName(spark, target_table)
            
            # Find records that have changed (e.g. country or gender changed)
            # We only look at records that are CURRENTLY active (is_current = True)
            changed_records = updates_df.alias("updates").join(
                spark.read.table(target_table).alias("target"),
                (F.col("updates.identifierHash") == F.col("target.identifierHash")) & 
                (F.col("target.is_current") == True)
            ).where(
                "updates.countrycode <> target.countrycode OR updates.gender <> target.gender"
            ).select("updates.identifierHash")

            # Expire the old records (Set is_current = False)
            if changed_records.count() > 0:
                print(f">>> Expiring {changed_records.count()} historical records...")
                change_ids = [row['identifierHash'] for row in changed_records.collect()]
                dt.update(
                    condition = F.col("identifierHash").isin(change_ids) & (F.col("is_current") == True),
                    set = { "is_current": "false", "effective_end_date": "current_timestamp()" }
                )

            # Append the new state
            updates_df.write.format("delta").mode("append").saveAsTable(target_table)
            
        else:
            # If table is empty, just do a clean initial load
            print(">>> Table is empty. Performing initial load...")
            updates_df.write.format("delta").mode("append").saveAsTable(target_table)
        
        # 6. LOG SUCCESS
        final_count = spark.read.table(target_table).count()
        log_pipeline_run(run_id, job_name, "Silver", "SUCCESS", final_count)
        print(f">>> [SUCCESS] {job_name} complete.")

    except Exception as e:
        print(f"!!! [ERROR] {job_name} failed: {str(e)}")
        log_pipeline_run(run_id, job_name, "Silver", "FAILED", 0, str(e))
        raise e

In [0]:
# PATH B: TRANSFORM REFERENCE DATA (Scheduled)
# ==========================================
if process_type in ["reference", "all"]:
    job_name = "Silver_Transform_Reference"
    try:
        print(f">>> [START] {job_name}...")
        total_records = 0
        
        # 1. Process Buyers
        buyersDF = spark.read.table("ecomm_data_project.bronze.buyers")
        int_cols = ['buyers', 'topbuyers', 'femalebuyers', 'malebuyers', 'totalproductsbought']
        
        for c in int_cols:
            buyersDF = buyersDF.withColumn(c, F.col(c).cast(IntegerType())).fillna(0, subset=[c])
            
        buyersDF = buyersDF.withColumn("country", F.initcap(F.col("country"))) \
                           .withColumn("female_to_male_ratio", F.round(F.col("femalebuyers") / (F.col("malebuyers") + 1), 2))
        
        buyersDF.write.format("delta").mode("overwrite").saveAsTable("ecomm_data_project.silver.buyers")
        total_records += buyersDF.count()

        # 2. Process Sellers & Countries
        for table in ["sellers", "countries"]:
            df = spark.read.table(f"ecomm_data_project.bronze.{table}")
            df.write.format("delta").mode("overwrite").saveAsTable(f"ecomm_data_project.silver.{table}")
            total_records += df.count()

        # Log Success
        log_pipeline_run(run_id, job_name, "Silver", "SUCCESS", total_records)
        print(f">>> [SUCCESS] {job_name} complete.")

    except Exception as e:
        log_pipeline_run(run_id, job_name, "Silver", "FAILED", 0, str(e))
        raise e